In [ ]:
!pip install nltk


In [11]:
from google.colab import files
uploaded = files.upload()


Saving True_Mid.csv to True_Mid.csv
Saving Fake_Mid.csv to Fake_Mid.csv


In [12]:
import pandas as pd

fake_df = pd.read_csv("Fake_Mid.csv")
real_df = pd.read_csv("True_Mid.csv")

# Combine and shuffle
data = pd.concat([fake_df, real_df], axis=0)
data = data.sample(frac=1).reset_index(drop=True)
data = data[['title', 'text', 'label']]
data.head()


,title,text,label
0,NASA Successfully Launches Lunar Mission,NASA's Artemis program has successfully launch...,1
1,NASA Successfully Launches Lunar Mission,NASA's Artemis program has successfully launch...,1
2,Shocking News: Aliens Found in USA!,A UFO has landed and aliens were discovered ne...,0
3,Shocking News: Aliens Found in USA!,A UFO has landed and aliens were discovered ne...,0
4,Shocking News: Aliens Found in USA!,A UFO has landed and aliens were discovered ne...,0


In [13]:
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Define cleaning function
def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('\\W', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

# Apply to the dataset
data['text'] = data['text'].apply(clean_text)
data.head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,title,text,label
0,NASA Successfully Launches Lunar Mission,nasa s artemis program has successfully launch...,1
1,NASA Successfully Launches Lunar Mission,nasa s artemis program has successfully launch...,1
2,Shocking News: Aliens Found in USA!,a ufo has landed and aliens were discovered ne...,0
3,Shocking News: Aliens Found in USA!,a ufo has landed and aliens were discovered ne...,0
4,Shocking News: Aliens Found in USA!,a ufo has landed and aliens were discovered ne...,0


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Inputs and labels
X = data['text']
y = data['label']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF
tfidf = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [15]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Train the model
model = PassiveAggressiveClassifier(max_iter=1000)
model.fit(X_train_tfidf, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 1.0
Confusion Matrix:
[[ 98   0]
 [  0 102]]


In [16]:
def predict_news(text):
    cleaned = clean_text(text)
    vectorized = tfidf.transform([cleaned])
    prediction = model.predict(vectorized)
    return "Real News" if prediction[0] == 1 else "Fake News"

# Example test
sample = "New vaccine discovered to treat rare viruses in children"
print("Prediction:", predict_news(sample))


Prediction: Fake News
